<a href="https://colab.research.google.com/github/MuhammadBilalH/Intent_Classification_with_LSTM/blob/main/FYP_UMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import sys
import spacy
import re
from gensim.models import KeyedVectors
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model

from tensorflow.keras.layers import Input
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense, Dropout,Activation
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.models import load_model
#from tensorflow.keras.utils.np_utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import GaussianNoise
from tensorflow.keras.regularizers import l2
from tensorflow.keras. callbacks import TensorBoard


In [ ]:
!pip install PyDrive

In [ ]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:

download = drive.CreateFile({'id': '1k7YYsJ_-lNG9o-MqNpSkFALq_uOG8vOm'})
download.GetContentFile('urduvec_140M_100K_300d.bin')

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving datasetCSVfull.csv to datasetCSVfull (1).csv


In [ ]:
df=pd.read_csv('datasetCSVfull.csv')
w2v = KeyedVectors.load_word2vec_format('urduvec_140M_100K_300d.bin', binary=True)

In [ ]:
df=df.sample(frac=1)
df.head()
X_train=df.iloc[:,0].values
Y_train=df.iloc[:,1].values


In [ ]:
def convert_to_vec(Y):
    list_of_vec=[]
    for sen in Y:
        if sen == 'OpenApp':
            list_of_vec.append(1)
        else:
            list_of_vec.append(0)
    return list_of_vec

In [ ]:
def pre_process_sentence(sentences):
    sentences = re.sub(r"\d+", " ", sentences)
    # English punctuations
    sentences = re.sub(r"""[!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~]+""", " ", sentences)
    # Urdu punctuations
    sentences = re.sub(r"[:؛؟’‘٭ء،۔]+", " ", sentences)
    # Arabic numbers
    sentences = re.sub(r"[٠‎١‎٢‎٣‎٤‎٥‎٦‎٧‎٨‎٩]+", " ", sentences)
    sentences = re.sub(r"[^\w\s]", " ", sentences)
    # Remove English characters and numbers.
    sentences = re.sub(r"[a-zA-z0-9]+", " ", sentences)
    # remove multiple spaces.
    sentences = re.sub(r" +", " ", sentences)
    return sentences

In [ ]:
def glove_vecs(X):
    tokenizer=spacy.blank('ur')
    list_of_X=[]
    for sen in X:
        sen=pre_process_sentence(sen)
        list_of_tokens=tokenizer(sen)
        list_of_token_vec=[]
        for token in list_of_tokens:
            try:
                tokenVec=w2v[""+token.text]
                
            except:
                tokenVec=np.zeros(300)
            list_of_token_vec.append(tokenVec)
        while len(list_of_token_vec)<20:
                    list_of_token_vec.append(np.zeros(300))
        #print(len(list_of_token_vec))
        #list_of_token_vec=list_of_token_vec/20
        #print(list_of_token_vec)
        list_of_X.append(list_of_token_vec)
    return list_of_X

In [ ]:
Y_train=convert_to_vec(Y_train)
X_train_vec=glove_vecs(X_train)
X_train_vec=np.array(X_train_vec)
train_size=int(len(X_train_vec)*0.8)
print(train_size)
X_new=X_train_vec[:train_size]
Y_new=Y_train[:train_size]
X_new_test=X_train_vec[train_size:]
Y_new_test=Y_train[train_size:]
print(len(X_new),len(Y_new))
batch_size=(len(X_new))
X_test=X_train_vec[train_size:]
Y_test=Y_train[train_size:]


1498
1498 1498


In [ ]:

model = Sequential()
model.add(LSTM(1155 , return_sequences=True,input_shape=(20, 300)))
model.add(LSTM(550, dropout = 0.4, recurrent_dropout=0.4))
model.add(Dense(1, kernel_regularizer=l2(0.2)))
model.add(Activation('sigmoid'))
model.summary()
adam = Adam(lr=0.0001)
model.compile(loss='binary_crossentropy', optimizer=adam,metrics = ['accuracy'] )
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 20, 1155)          6726720   
_________________________________________________________________
lstm_1 (LSTM)                (None, 550)               3753200   
_________________________________________________________________
dense (Dense)                (None, 1)                 551       
_________________________________________________________________
activation (Activation)      (None, 1)                 0         
Total params: 10,480,471
Trainable params: 10,480,471
Non-trainable params: 0
_________________________________________________________________
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 20, 1155)          6726720   
________________________

In [ ]:
model_history = model.fit(x=np.array(X_new), y=np.array(Y_new), batch_size=110, epochs=55, verbose=1,validation_data = (np.array(X_new_test), np.array(Y_new_test)))

Epoch 1/55
14/14 [==============================] - 4s 251ms/step - loss: 0.0290 - accuracy: 0.9960 - val_loss: 0.0762 - val_accuracy: 0.9867
Epoch 2/55
14/14 [==============================] - 3s 248ms/step - loss: 0.0290 - accuracy: 0.9960 - val_loss: 0.0760 - val_accuracy: 0.9867
Epoch 3/55
14/14 [==============================] - 3s 243ms/step - loss: 0.0290 - accuracy: 0.9967 - val_loss: 0.0758 - val_accuracy: 0.9867
Epoch 4/55
14/14 [==============================] - 3s 250ms/step - loss: 0.0290 - accuracy: 0.9960 - val_loss: 0.0780 - val_accuracy: 0.9867
Epoch 5/55
14/14 [==============================] - 3s 245ms/step - loss: 0.0291 - accuracy: 0.9953 - val_loss: 0.0816 - val_accuracy: 0.9840
Epoch 6/55
14/14 [==============================] - 3s 242ms/step - loss: 0.0290 - accuracy: 0.9960 - val_loss: 0.0747 - val_accuracy: 0.9867
Epoch 7/55
14/14 [==============================] - 3s 246ms/step - loss: 0.0291 - accuracy: 0.9960 - val_loss: 0.0750 - val_accuracy: 0.9867
Epoch 

In [ ]:
from tensorflow import lite


In [ ]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /tmp/tmp3en4qy2z/assets
